<a href="https://colab.research.google.com/github/layoncezar/COVID-19/blob/master/TCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Definindo a amostra
pacotes <- c("plotly","tidyverse","ggrepel","fastDummies","knitr","kableExtra",
             "splines","reshape2","PerformanceAnalytics","correlation","see",
             "ggraph","psych","nortest","rgl","car","ggside","tidyquant","olsrr",
             "jtools","ggstance","magick","cowplot","emojifont","beepr","Rcpp",
             "equatiomatic")

options(rgl.debug = TRUE)

if(sum(as.numeric(!pacotes %in% installed.packages())) != 0){
  instalador <- pacotes[!pacotes %in% installed.packages()]
  for(i in 1:length(instalador)) {
    install.packages(instalador, dependencies = T)
    break()}
  sapply(pacotes, require, character = T) 
} else {
  sapply(pacotes, require, character = T) 
}


# dados de morados
Morador<-readRDS("MORADOR.rds")

#Criando um id para cada linha
#id<-cod_upa*100+num_dom
id<-Morador$COD_UPA*100+Morador$NUM_DOM
id<-as.data.frame(id)

#Contando quantos ids estão duplicados e qual é o número de ids(pessoas) por domicílio

df_unique <- subset(id, !duplicated(id))
table(id)

df_count <- id %>% group_by(id) %>% summarize(count = n())

#Base que fica com o id
Base_morador<-transmute(Morador, id, UF, TIPO_SITUACAO_REG, V0403, V0404, V0405, V0409, V0410,
                        V0411, V0413, V0306,ANOS_ESTUDO, RENDA_TOTAL, PC_RENDA_DISP)

#Juntando as bases com base no id comum no intuito de inserir a quantidade de moradores por id
#Juntando as duas bases
Base_morador2<-merge(Base_morador, df_count, by.x="id")

#Apenas dados do chefe de família
BaseMorador3<-Base_morador2[Base_morador2$V0306==1, ]

#Apenas dados quanti
BaseMoradorQuanti<-select(BaseMorador3, "V0403", "V0409", "V0410", "V0411",
                          "V0413", "ANOS_ESTUDO", "PC_RENDA_DISP", "count")

#Correlação entre variáveis
library(corrplot)
a<-cor(BaseMoradorQuanti, method = c("pearson"))
corrplot(a, method = "number")

#Conclusão inicial: Baixa correlação entre as variáveis

#############A base acima é a base total com dados quali e quanti apenas relacionados ao morador####

### Extraindo dados de consumo alimentar #####
#Leitura dos dados em rds
Consumo<-readRDS("CONSUMO_ALIMENTAR.rds")

#Selecionando variáveis
Consumo_select<-select(Consumo, "UF", "TIPO_SITUACAO_REG", 
                       "COD_UPA", "NUM_DOM", "V9001","QTD", "PESO_FINAL")

#Se eu for usar uma análise discriminante ou regressão logística aqui eu devo criar
#uma nova base de dados. Considerando o consumo ou não. Daqui para baixo é só de
#consumidores de café

#Apenas Café 
Consumo_cafe<-Consumo_select[Consumo_select$V9001>=8501302 & Consumo_select$V9001<=8501311,]
dim(Consumo_cafe)
valores<-Consumo_cafe$V9001
valores<-as.data.frame(valores)
summary(valores)
rotulos<-c("Cafe", "Cafe com Leite", "Cafe espresso", "Capuccino", "Capuccino Desc", 
           "Capuccino Light", "Carioca", "Cafe sub Leite", "Cafe Desc", "Cafe esp Desc")


fator_coluna <- factor(valores, levels = 8501302:8501311, labels = rotulos)
fator_coluna<-as.data.frame(fator_coluna)
summary(fator_coluna)
tabelacafe<-table(fator_coluna)
tabelacafe<-as.data.frame(tabelacafe)

install.packages("treemap")
library(treemap)
treemap(tabelacafe, 
        index = "fator_coluna", 
        vSize = "Freq", 
        type = "index", 
        title = "Meu Gráfico Treemap")



#Criando um id para cada linha
#id<-cod_upa*100+num_dom
id2<-Consumo_cafe$COD_UPA*100+Consumo_cafe$NUM_DOM
id2<-as.data.frame(id2)


#Juntando o id ao Bando de Dados
#O transmute permite selecionar as bases que serão mantidas ou não e a nova coluna a ser incorporada
Base_c<-transmute(Consumo_cafe,id2, UF, TIPO_SITUACAO_REG, V9001,QTD, PESO_FINAL)
#Usa essa Base_C (daqui para frente) para separar os demais grupos de consumidores

#juntando a base a partir da soma da quantidade de cada família

#reduzir id pela soma de todas as variáveis
dados_reduzidos <- aggregate(QTD ~ id2, data = Base_c, FUN = sum)

#Juntando o peso a dados reduzidos
Base_Peso<-transmute(Base_c, id2, PESO_FINAL)
dim(Base_Peso)

#Aqui eu só tenho as variáveis que vou usar na regressão geral (Quantidade). Se eu for usar os dados de cada grupo de 
#produto, eu tenho que usar os códigos de cada tipo de café

#Juntando as duas bases
BaseGeral<- merge(BaseMorador3,dados_reduzidos, by.x = "id", by.y = "id2")
Geral2<- merge(BaseGeral, Base_Peso, by.x = "id", by.y = "id2")
dim(BaseGeral)

#Incluindo o peso final
library(dplyr)
Base_Final <- Geral2 %>% 
  group_by(id, PESO_FINAL) %>% 
  distinct() %>% 
  ungroup()

#Mudando o nome das variáveis
names(Base_Final)<-c("id", "UF", "TIPO_SITUACAO_REG","Idade_chefe", "Sexo_chefe",
                     "Cor_Chefe", "Qtd_cartao", "Qtd_CC", "Qtd_cheque", "Qtd_poup", "ref_UC", 
                     "Anos_estudo", "Renda_total", "Renda_per_c", "Qtd_morador", "Qtd_Cons", "PESO")
sum(Base_Final$PESO)
dim(Base_Final)
summary(Base_Final)

#--------------------------------------------------------------------------------
#Análise de cluster
#------------------Análise de Cluster por K-means----------
#instalar pacotes
install.packages("FactorMineR")
install.packages("factoextra")
install.packages("cluster")
install.packages("openxlsx")
install.packages("readxl")
install.packages("AER")
install.packages("writexl")

#Chamar os pacotes
library(FactoMineR)
library(factoextra)
library(cluster)
library(openxlsx)
library(readxl)
library(AER)
library(writexl)


#Base sem Z-score e sem dados faltantes #Base a ser incorporado os clusters
dados_gerais<-na.omit(QuantiRaiz)
dados_gerais<-as.data.frame(dados_gerais)

#Estatística descritiva dos dados
summary(dados_gerais)
table(dados_gerais$TIPO_SITUACAO_REG)
prop.table(table(dados_gerais$Cor_Chefe))
sd(dados_gerais$Qtd_Cons)
dados_gerais$Qtd_Cons<-exp(dados_gerais$Qtd_Cons)

#Transformar em Escala e identificar outliers e remover NA
z_score<-scale(QuantiCluster2)
summary(z_score)

#(dados faltantes) 
dados_brutos<-na.omit(z_score)
dados_brutos<-as.data.frame(dados_brutos)

#Verificando a presença de outliers
boxplot(dados_brutos)
summary(dados_brutos)

#Definir a quantidade de Cluster
fviz_nbclust(dados_brutos, kmeans, method = "silhouette")
fviz_nbclust(dados_brutos, kmeans, method = "gap_stat")
fviz_nbclust(dados_brutos, kmeans, method = "wss")

# Número ótimo de clusters
library(factoextra)

fviz_nbclust(dados_brutos, kmeans, method = "wss")+
  geom_vline(xintercept = 4, linetype = 2)

#Gerar Kmeans
dados_kmeans<-kmeans(dados_brutos, 2)
dados_kmeans2<-kmeans(dados_brutos, 3)

#Visualizacao do grafico
fviz_cluster(dados_kmeans, data=dados_brutos, ellipse.type = "euclid", geom = "point")
fviz_cluster(dados_kmeans2, data=dados_brutos, ellipse.type = "euclid", geom = c("point", "text") )
?fviz_cluster

#criando uma lista com o cluster
lista<-dados_kmeans$cluster
lista

#Agrupando em uma tabela
dados_gerais<-cbind(dados_gerais,lista)
dim(dados_gerais)

#Analisando as características de cada cluster
#Lista 1
Lista1<-subset(dados_gerais, dados_gerais$lista=="1")
dim(Lista1)
Lista1$Qtd_Cons<-exp(Lista1$Qtd_Cons)

summary(Lista1)

#Lista 2
Lista2<- subset(dados_gerais, dados_gerais$lista=="2")
dim(Lista2)
Lista2$Qtd_Cons<-exp(Lista2$Qtd_Cons)
summary(Lista2)


#Exportando as tabelas em csv

write.table(dados_gerais, "c:saida.csv", sep="," )
write_xlsx(dados_gerais, "c:saida.xlsx")


#--------------------------------------------------------
#Regressão Geral
#Pacotes da regressao
pacotes <- c("plotly","tidyverse","ggrepel","fastDummies","knitr","kableExtra",
             "splines","reshape2","PerformanceAnalytics","correlation","see",
             "ggraph","psych","nortest","rgl","car","ggside","tidyquant","olsrr",
             "jtools","ggstance","magick","cowplot","emojifont","beepr","Rcpp",
             "equatiomatic")

options(rgl.debug = TRUE)

if(sum(as.numeric(!pacotes %in% installed.packages())) != 0){
  instalador <- pacotes[!pacotes %in% installed.packages()]
  for(i in 1:length(instalador)) {
    install.packages(instalador, dependencies = T)
    break()}
  sapply(pacotes, require, character = T) 
} else {
  sapply(pacotes, require, character = T) 
}

#Removendo colunas específicas
dados_gerais$Qtd_Cons<- exp(dados_gerais$Qtd_Cons)
dados <- subset(dados_gerais, select = -c(Renda_total))

#Verificar normalidade da variável dependente
hist(dados$Qtd_Cons)

#Como não é normal vou realizar uma transformação de box-cox
#Para calcular o lambda de Box-Cox
#função 'powerTransform' do pacote 'car'
lambda_BC <- powerTransform(dados$Qtd_Cons)
lambda_BC

#Inserindo o lambda de Box-Cox na base de dados para a estimação de um novo modelo
dados$Qtd_Cons <- (((dados$Qtd_Cons ^ lambda_BC$lambda) - 1) / 
                     lambda_BC$lambda)

#Verificar normalidade novamente
hist(dados$Qtd_Cons)

#Testando os modelos
Modelo2<-lm(Qtd_Cons~., data=dados)
summary(Modelo2)

#Aplicando log em renda per capita
dados$Renda_per_c<-log(dados$Renda_per_c)
hist(dados$Renda_per_c)
dados2<-na.omit(dados)
dim(dados)
summary(dados2)

#Novo Modelo
ModeloNA<-lm(Qtd_Cons~., data=dados2)
summary(ModeloNA)

#Verificando os resíduos do modelo
residuos <- residuals(Modelo2)
sd_residuos <- sd(residuos)
limite_superior <- mean(residuos) + 3 * sd_residuos
limite_inferior <- mean(residuos) - 3 * sd_residuos
outliers <- which(residuos > limite_superior | residuos < limite_inferior)
print(outliers)

#Normalidade
library(nortest)
res<-ModeloNA$residuals
qqnorm(res)
qqline(res)
hist(ModeloNA$residuals)
#OK, Distribuição normal agora

#Heterocedasticidade
ols_test_breusch_pagan(ModeloNA)

#Passo 1 dumizar
dummy1<-dummy_columns(.data = dados2,
                      select_columns = "UF",
                      remove_selected_columns = T,
                      remove_most_frequent_dummy = T)
dummy2<-dummy_columns(.data = dummy1,
                      select_columns = "TIPO_SITUACAO_REG",
                      remove_selected_columns = T,
                      remove_most_frequent_dummy = T)
dummy3<-dummy_columns(.data = dummy2,
                      select_columns = "Sexo_chefe",
                      remove_selected_columns = T,
                      remove_most_frequent_dummy = T)
dummy4<-dummy_columns(.data = dummy3,
                      select_columns = "Cor_Chefe",
                      remove_selected_columns = T,
                      remove_most_frequent_dummy = T)

#Modelo após dumizadação
Modelodumizado<-lm(Qtd_Cons~., data=dummy4)
summary(Modelodumizado)

#Após a identificação dos outliers deve-se excluir as linhas
linhas_para_excluir <- c(446, 445, 502, 501, 1484, 1482, 1823, 1821, 1898,
                         1896, 2043, 2041, 2063, 2061, 4314, 4310, 4659, 4655,
                         4818, 4814, 6057, 6049)

# excluir as linhas do dataframe
df_sem_linhas <- subset(dummy4, !row.names(dummy4) %in% linhas_para_excluir)
print(df_sem_linhas)

#Rodando outro modelo com esse novo dataset
Modelodf<-lm(Qtd_Cons~., data=df_sem_linhas)
summary(Modelodf)

#Verificando heterocedasticidade no modelo
ols_test_breusch_pagan(Modelodf)
boxplot(df_sem_linhas)

#Verificando Multicolinearidade
ols_vif_tol(Modelodf)

#Gráfico de resíduos
residuos <- Modelodf$residuals
valores_ajustados <- Modelodf$fitted.values
plot(valores_ajustados, residuos, main = "Gráfico de Resíduos vs. Valores Ajustados", 
     xlab = "Valores Ajustados", ylab = "Resíduos")

abline(h = 0, col = "red")
title(xlab = "Valores Ajustados", ylab = "Resíduos")


#Stepwise
step_modelo_bc <- step(Modelodf, k = 3.841459)
summary(step_modelo_bc)

#Melhor modelo agora é esse chamado Modelodf

#Visualizando a equação da regressão
extract_eq(step_modelo_bc, use_coefs = T) %>%
  kable() %>%
  kable_styling(bootstrap_options = "striped",
                full_width = F,
                font_size = 10)

#Outras maneiras de apresentar os outputs do modelo
#função 'summ' do pacote 'jtools'

ap1<-summ(Modelodf, confint = T, digits = 4, ci.width = .95)
ap1<-tidy(ap1)
ap2<-summ(step_modelo_bc, confint = T, digits = 4, ci.width = .95)
ap2<-tidy(ap2)

#Comparando modelos
export_summs(Modelodf, step_modelo_bc, scale=F)

#Visualizando a equação da regressão
extract_eq(step_modelo_bc, use_coefs = T) %>%
  kable() %>%
  kable_styling(bootstrap_options = "striped",
                full_width = F,
                font_size = 20
  )

# exportar o dataframe como um arquivo xlsx
library(openxlsx)

write.xlsx(ap1, "ap1.xlsx")
write.xlsx(ap2, "ap2.xlsx")

#Deu certo!!!!!

#Citação do R no TCC
citation()
#---------------------------------------------------------

SyntaxError: ignored